<a href="https://colab.research.google.com/github/ldennis1/BSAN6070--CA06/blob/main/CA06_kNN_Based_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Load the data and perform EDA 

In [20]:
#import necessary packages 
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

In [21]:
#read in data
movies = pd.read_csv('https://github.com/ArinB/CA05-kNN/raw/master/movies_recommendation_data.csv')

In [22]:
#view top of data
movies.head()

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0,0


In [23]:
#view end of data
movies.tail()

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
25,17,Thor: Ragnarok,7.9,0,0,0,1,0,0,0,0
26,12,Spirit: Stallion of the Cimarron,7.1,0,0,0,0,0,0,0,0
27,1,Hacksaw Ridge,8.2,1,1,0,0,0,0,1,0
28,86,12 Years a Slave,8.1,1,1,0,0,0,0,1,0
29,46,Queen of Katwe,7.4,1,1,0,0,0,0,0,0


In [24]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie ID     30 non-null     int64  
 1   Movie Name   30 non-null     object 
 2   IMDB Rating  30 non-null     float64
 3   Biography    30 non-null     int64  
 4   Drama        30 non-null     int64  
 5   Thriller     30 non-null     int64  
 6   Comedy       30 non-null     int64  
 7   Crime        30 non-null     int64  
 8   Mystery      30 non-null     int64  
 9   History      30 non-null     int64  
 10  Label        30 non-null     int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 2.7+ KB


In [25]:
movies.describe()

,Movie ID,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.0
mean,48.133333,7.696667,0.233333,0.600000,0.100000,0.100000,0.133333,0.100000,0.100000,0.0
std,29.288969,0.666169,0.430183,0.498273,0.305129,0.305129,0.345746,0.305129,0.305129,0.0
min,1.000000,5.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,27.750000,7.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,48.500000,7.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,64.250000,8.175000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,98.000000,8.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [26]:
movies.drop('Label', axis = 1, inplace = True)
movies

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0
5,98,21,6.8,0,1,0,0,1,0,1
6,31,Gifted,7.6,0,1,0,0,0,0,0
7,3,Travelling Salesman,5.9,0,1,0,0,0,1,0
8,51,Avatar,7.9,0,0,0,0,0,0,0
9,47,The Karate Kid,7.2,0,1,0,0,0,0,0


In [27]:
movies.isnull().sum()

Movie ID       0
Movie Name     0
IMDB Rating    0
Biography      0
Drama          0
Thriller       0
Comedy         0
Crime          0
Mystery        0
History        0
dtype: int64

# Step 2: Build our recommender system 

In [30]:
#get column names in a list 
movies.columns

Index(['Movie ID', 'Movie Name', 'IMDB Rating', 'Biography', 'Drama',
       'Thriller', 'Comedy', 'Crime', 'Mystery', 'History'],
      dtype='object')

In [35]:
#define our variables 
x = movies[['IMDB Rating', 'Biography', 'Drama',
       'Thriller', 'Comedy', 'Crime', 'Mystery', 'History']]
y = movies['Movie Name']

In [ ]:
#implement nearestneighbors model
nbrs = NearestNeighbors(n_neighbors=5).fit(x)
distances, indices = nbrs.kneighbors(x)
print(distances)

Genre Information: "The Post"

IMDB Rating = 7.2, Biography = Yes, Drama = Yes, Thriller = No, Comedy = No, Crime = No,
Mystery = No, History = Yes

In [50]:
#define variable for features of "The Post"
the_post = [7.2,1,1,0,0,0,0,1]

distances, indices = nbrs.kneighbors([the_post])
print(indices)

[[28 27 29 16  2]]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


In [51]:
#return movie titles of recommended movies
for title in indices:
  print("Recommended Movies:")
  print(movies.iloc[title]['Movie Name'])
  print("*Enjoy!*")

Recommended Movies:
28    12 Years a Slave
27       Hacksaw Ridge
29      Queen of Katwe
16      The Wind Rises
2     A Beautiful Mind
Name: Movie Name, dtype: object
*Enjoy!*
